In [1]:
import os
import glob
import numpy as np
from tqdm import tqdm, trange
from scipy.signal import butter, filtfilt, savgol_filter
import matplotlib.pyplot as plt
from ipywidgets import interact
import onnxruntime as ort
import matplotlib.pyplot as plt
from tqdm import trange
import pandas as pd

In [2]:
# extract data from txt file and store in a 3D array
# the output array will have the shape (n_files, signal_length, channels)
# the 3 channel values are following:
# 1:Dem-Dexc   2:Aem-Dexc       3:E


import numpy as np
import os
import glob
from tqdm import tqdm
import pandas as pd

def extract_values_from_file(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()
    
    # Skip the header lines until the actual data starts
    data_start_index = 0
    for i, line in enumerate(lines):
        if line.strip().startswith("DemDexc"):
            data_start_index = i + 1
            break
    
    # Extract Dem-Dexc, Aem-Dexc, and E values
    values = []
    for line in lines[data_start_index:]:
        parts = line.split()
        if len(parts) >= 3:  # Ensure there are at least three columns of data
            dem_dexc = float(parts[0])
            aem_dexc = float(parts[1])
            E = float(parts[2])
            values.append([dem_dexc, aem_dexc, E])
    
    return np.array(values)

folder_path = 'SRP_proximal_length/oligo/kinetics'  # Path containing the txt files
output_dir = folder_path + '/output'
os.makedirs(output_dir, exist_ok=True)

file_paths = glob.glob(os.path.join(folder_path, '*.txt'))
file_paths.sort()
array_list = []
file_names = []

for file_path in tqdm(file_paths):
    values_2d = extract_values_from_file(file_path)
    array_list.append(values_2d)
    file_names.append(os.path.basename(file_path))  # Store the file name

data_3d = np.array(array_list)  # Convert the list of 2D arrays to a 3D numpy array
# Print the shape of the 3D array to verify
print(data_3d.shape)


# Save the file names to a pandas DataFrame and export it
file_names_df = pd.DataFrame(file_names, columns=['FileName'])




# Save the 3D array to a file
#np.save(output_dir + '/FRET_raw.npy', data_3d) # Save the 3D array to a file

100%|██████████████████████████████████████████| 64/64 [00:00<00:00, 525.83it/s]

(64, 800, 3)


In [3]:
# load np file if needed
#data_3d = np.load(output_dir + '/FRET_raw.npy') # change the path if needed

def normalize(matrix, axis=1, new_min=0, new_max=1):
    # Extract min and max along the specified axis, keeping dimensions for broadcasting
    arr_min = np.min(matrix, axis=axis, keepdims=True)
    arr_max = np.max(matrix, axis=axis, keepdims=True)
    
    # Prevent division by zero
    scale = arr_max - arr_min
    scale[scale == 0] = 1  # Prevent division by zero (if max equals min)

    # Normalize along the specified axis
    normalized = (matrix - arr_min) / scale  # Scale to [0, 1]
    scaled = normalized * (new_max - new_min) + new_min  # Scale to [new_min, new_max]
    
    return scaled


# Apply Butterworth low-pass filter
def apply_low_pass_filter(data_column):
    b, a = butter(N=5, Wn=cutoff_frequency, btype='low')
    filtered_data = filtfilt(b, a, data_column)
    return filtered_data


def apply_low_pass_filter_3d(data, cutoff_frequency, axis=1):
    # Parameters for the Butterworth filter
    N = 5  # Order of the filter
    b, a = butter(N=N, Wn=cutoff_frequency, btype='low') # fs is the sampling frequency

    # Initialize the filtered data array
    filtered_data = np.zeros_like(data)
    
    # Apply the filter along the specified axis
    if axis == 1:
        for i in range(data.shape[0]):
            for j in range(data.shape[2]):
                filtered_data[i, :, j] = filtfilt(b, a, data[i, :, j])
    return filtered_data



def apply_low_pass_and_smooth_3d(data, cutoff_frequency, window_size, axis=1):
    # Parameters for the Butterworth filter
    N = 5  # Order of the filter
    b, a = butter(N=N, Wn=cutoff_frequency, btype='low')  # fs is the sampling frequency

    # Initialize the filtered and smoothed data array
    filtered_data = np.zeros_like(data)
    smoothed_data = np.zeros_like(data)
    
    # Apply the filter along the specified axis
    if axis == 1:
        for i in range(data.shape[0]):
            for j in range(data.shape[2]):
                filtered_row = filtfilt(b, a, data[i, :, j])
                filtered_data[i, :, j] = filtered_row
                # Apply Savitzky-Golay filter for smoothing
                smoothed_data[i, :, j] = savgol_filter(filtered_row, window_size, 2)
                
    return smoothed_data


# Parameters for the low-pass filter
cutoff_frequency = 0.2  # Adjust the cutoff frequency as needed
window_size = 3  # Adjust the window size for moving average or Savitzky-Golay filter

data_norm = normalize(data_3d)
data_norm_filtered = apply_low_pass_and_smooth_3d(data_norm, cutoff_frequency, window_size, axis=1)

#np.save(output_dir + '/FRET_norm.npy', data_norm) # Save the 3D array to a file
#np.save(output_dir + '/FRET_norm_filtered.npy', data_norm_filtered) # Save the 3D array to a file


In [4]:
onnx_path = "model2.onnx" #path of the model
session = ort.InferenceSession(onnx_path)

input_name = session.get_inputs()[0].name
input_data = data_norm.astype(np.float32) # 3D array

outputs = session.run(None, {input_name: input_data})
y_pred = outputs[0] # 3D array

In [5]:
y_pred.shape, data_norm.shape, data_norm_filtered.shape, data_3d.shape

((64, 800, 6), (64, 800, 3), (64, 800, 3), (64, 800, 3))

In [6]:
E_FRET = data_3d[:, :, 2]
E_FRET[E_FRET < 0] = 0   # Set elements less than 0 to 0
E_FRET[E_FRET > 1] = 1   # Set elements greater than 1 to 1

time = np.linspace(0.05, 0.05 * data_norm.shape[1], data_norm.shape[1])


In [11]:

def update_plot1(i):
    plt.figure(figsize=(14, 10), dpi=300)
    
    # First subplot
    plt.subplot(2, 1, 1)  # (rows, cols, index)
    plt.plot(time, data_norm[i,:,0], color='lightgreen', alpha = 0.5)
    plt.plot(time, data_norm[i,:,1], color='#FFAAAA', alpha = 0.5)
    plt.plot(time, data_norm_filtered[i,:,1], label='A', color='red', alpha = 0.7)
    plt.plot(time, data_norm_filtered[i,:,0], label='D', color='green', alpha = 0.7)
    plt.plot(time, y_pred[i,:, 0], color = "black", label = "dyn", linewidth = 3)
    plt.plot(time, y_pred[i,:, 1], color = "brown", label = "static", linewidth = 3)
    plt.plot(time, y_pred[i,:, 2], color = "fuchsia", label = "scrambled")
    plt.plot(time, y_pred[i,:, 3], color = "blue", label = "noisy")
    
    plt.plot(time, y_pred[i,:, 4], color = "violet", label = "aggregate")
    plt.plot(time, y_pred[i,:, 5], color = "orange", label = "bleached")
    plt.legend()
    
    plt.xlabel('T (s)')
    plt.ylabel('Values')

    plt.title(f"{file_names_df.loc[i, 'FileName']}")
    plt.legend(loc='upper right', prop={'size': 8})
    plt.grid()

    # Second subplot (left y-axis for raw data, right y-axis for E)
    ax1 = plt.subplot(2, 1, 2)  # (rows, cols, index)
    ax1.plot(time, data_3d[i, :, 0], label='DemDexc (raw)', color='green')
    ax1.plot(time, data_3d[i, :, 1], label='AemDexc (raw)', color='red')
    ax1.set_xlabel('T (s)')
    ax1.set_ylabel('Raw Intensity')
    ax1.legend(loc='upper left')
    ax1.grid()

    ax2 = ax1.twinx()
    ax2.plot(time, E_FRET[i, :], label='E', color='blue')
    ax2.set_ylabel('E Intensity')
    ax2.legend(loc='upper right')
    ax2.grid()

    plt.tight_layout()
    plt.show()

interact(update_plot1, i=(0, data_norm.shape[0] - 1, 1))


interactive(children=(IntSlider(value=31, description='i', max=63), Output()), _dom_classes=('widget-interact'…

<function __main__.update_plot1(i)>